<a href="https://colab.research.google.com/github/runnithan03/Dissertation/blob/main/Code/Filling_Missing_Values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import os

uploaded = files.upload()

uploaded_filename = list(uploaded.keys())[0]
os.rename(uploaded_filename, 'Profit ESG dataset.csv')  # Rename it to 'Profit ESG dataset.csv'

Saving Profit ESG dataset.csv to Profit ESG dataset (1).csv


In [ ]:
%reload_ext rpy2.ipython


In [ ]:
%%R

data <- read.csv('Profit ESG dataset.csv')
data$equity_category <- as.factor(data$equity_category)
data$rating <- as.factor(data$rating)
data$risk_rating <- as.factor(data$risk_rating)

head(data)

    roe sustainability_score                         equity_category rating
1 22.69                22.44                      Switzerland Equity      3
2 16.14                20.52                           Sweden Equity      2
3 14.88                18.91 Sector Equity Consumer Goods & Services      3
4 10.04                20.73               Eurozone Large-Cap Equity      4
5  8.75                23.96                  Japan Large-Cap Equity      3
6 31.07                22.84           Global Large-Cap Blend Equity      5
  risk_rating equity_size_score price_cash_flow_ratio dividend_yield_factor
1           3            328.15                 14.80                  3.23
2           2            248.09                 10.26                  1.45
3           3            308.52                 14.41                  2.59
4           3            275.97                  7.89                  2.60
5           2            286.08                  8.40                  2.75
6           

In [ ]:
%%R

table(rowSums(is.na(data)))


   0    1    2    3    4    8    9   10   11 
2886 1004 1377  287   16  268    5   74    4 


In [ ]:
%%R

# Filter rows with less than 3 NA values
clean_data <- data[rowSums(is.na(data)) < 3, ]
str(clean_data)

'data.frame':	5267 obs. of  14 variables:
 $ roe                       : num  22.69 16.14 14.88 10.04 8.75 ...
 $ sustainability_score      : num  22.4 20.5 18.9 20.7 24 ...
 $ equity_category           : Factor w/ 101 levels "Africa & Middle East Equity",..: 86 84 68 27 52 40 96 93 86 72 ...
 $ rating                    : Factor w/ 5 levels "1","2","3","4",..: 3 2 3 4 3 5 3 3 2 3 ...
 $ risk_rating               : Factor w/ 5 levels "1","2","3","4",..: 3 2 3 3 2 3 2 2 4 2 ...
 $ equity_size_score         : num  328 248 309 276 286 ...
 $ price_cash_flow_ratio     : num  14.8 10.26 14.41 7.89 8.4 ...
 $ dividend_yield_factor     : num  3.23 1.45 2.59 2.6 2.75 1.88 2.46 4.08 3.34 2.65 ...
 $ historical_earnings_growth: num  6.24 8.97 -10.14 -1.07 -3.99 ...
 $ sales_growth              : num  1.25 2.66 5.86 0.7 3.52 7.5 6.41 1.81 1.4 3.95 ...
 $ asset_cash                : num  0.19 0.01 0.54 0.7 0.37 0 0 1.01 1.25 0.76 ...
 $ holdings_n_stock          : int  20 30 34 248 320 NA NA 99 30

In [ ]:
%%R
install.packages("ranger")
library(ranger)
library(dplyr)

# Function to impute missing continuous variables using lm
impute_continuous_lm <- function(data, column) {
  # Train a linear model excluding rows with NA in the column
  lm_model <- lm(as.formula(paste(column, "~ .")), data = data, na.action = na.exclude)

  # Predict missing values
  missing_indices <- which(is.na(data[[column]]))
  if (length(missing_indices) > 0) {
    predictions <- predict(lm_model, newdata = data[missing_indices, ])

    # Replace NA values with predictions
    data[[column]][missing_indices] <- predictions
  }

  return(data)
}

# Function to impute missing categorical variables using ranger
impute_categorical_rf <- function(data, column) {
  # Train a Random Forest classification model excluding rows with NA in the column
  rf_model <- ranger(
    formula = as.formula(paste(column, "~ .")),
    data = data[!is.na(data[[column]]), ],
    na.action = "na.omit",
    classification = TRUE
  )

  # Predict missing values
  missing_indices <- which(is.na(data[[column]]))
  if (length(missing_indices) > 0) {
    predictions <- predict(
      rf_model,
      data = data[missing_indices, ]
    )$predictions

    # Replace NA values with predictions
    data[[column]][missing_indices] <- predictions
  }

  return(data)
}

# Identify numerical and categorical columns
numerical_columns <- names(clean_data)[sapply(clean_data, is.numeric)]
categorical_columns <- names(clean_data)[sapply(clean_data, is.factor)]

# Impute missing values for numerical columns using lm
for (col in numerical_columns) {
  if (any(is.na(clean_data[[col]]))) {
    clean_data <- impute_continuous_lm(clean_data, col)
  }
}

# Impute missing values for categorical columns using ranger
for (col in categorical_columns) {
  if (any(is.na(clean_data[[col]]))) {
    clean_data <- impute_categorical_rf(clean_data, col)
  }
}

# Verify that no missing values remain
if (any(is.na(clean_data))) {
  stop("There are still NA values in the data!")
} else {
  print("All missing values have been successfully imputed!")
}

# Summary of the cleaned dataset
summary(clean_data)

# View the first few rows of the cleaned dataset
# head(clean_data)

any(is.na(clean_data)) # Should return FALSE

(as ‘lib’ is unspecified)







	‘/tmp/RtmpNlqoYo/downloaded_packages’



[1] "All missing values have been successfully imputed!"
[1] FALSE


In [ ]:
%%R
clean_data$holdings_n_stock <- round(clean_data$holdings_n_stock) # holdings_n_stock is an integer value
clean_data$price_cash_flow_ratio <- signif(clean_data$price_cash_flow_ratio, digits = 3)


# Save the new dataset as a CSV file
write.csv(clean_data, "clean_data.csv", row.names = FALSE)

In [ ]:
from google.colab import files
files.download("clean_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>